In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dask.distributed import Client, LocalCluster

client = Client() # Note that `memory_limit` is the limit **per worker**.
# n_workers=4,
#                 threads_per_worker=1,
#                 memory_limit='3GB'
client # If you click the dashboard link in the output, you can monitor real-time progress and get other cool visualizations.

In [ ]:
import copy
import sys
import xarray as xr
import numpy as np
import dask.array as da

import matplotlib.pyplot as plt
import hvplot.xarray
import scipy.constants
import scipy.signal as signal
import pickle
import holoviews as hv

sys.path.append("..")
import processing_dask as pr
import plot_dask
import processing as old_processing

sys.path.append("../../preprocessing/")
from generate_chirp import generate_chirp

### Open and resave file

In [ ]:
# file path to data and configs

prefix = "/Users/abroome/Desktop/McMurdo2022/12042022_anna/20221204_170000" # McMurdo Ice Shelf/Eastwind Glacier grounding zone

# resave data as zarr for dask processing
zarr_path = pr.save_radar_data_to_zarr(prefix)

# open zarr file, adjust chunk size to be 10 MB - 1 GB based on sample rate/bit depth
raw = xr.open_zarr(zarr_path, chunks={"pulse_idx": 1000})


### Enter processing parameters

In [ ]:
# choose zero sample index based on sample rate (for X310 only!)
if raw.config['GENERATE']['sample_rate'] == 20e6:
    zero_sample_idx = 36
elif raw.config['GENERATE']['sample_rate'] == 50e6:
    zero_sample_idx = 63
elif raw.config['GENERATE']['sample_rate'] == 100e6:
    zero_sample_idx = 81
else:
    print("different bandwidth, need to specify zero sample index")
#zero_sample_idx = 159 # B205mini, fs = 56 MHz
#zero_sample_idx = 166 # B205mini, fs = 20 MHz

nstack = 10 # number of pulses to stack #len(raw.pulse_idx) 

modify_rx_window = False # set to true if you want to window the reference chirp only on receive, false uses ref chirp as transmitted in config file
rx_window = "kaiser14" # what you want to change the rx window to if modify_rx_window is true

dielectric_constant = 3.17 # ice (air = 1, 66% velocity coax = 2.2957)
#dielectric_constant = 2.2957 # COAX (air = 1, 66% velocity coax = 2.2957)
sig_speed = scipy.constants.c / np.sqrt(dielectric_constant)

### Generate reference chirp

In [ ]:
if modify_rx_window:
    config = copy.deepcopy(raw.config)
    config['GENERATE']['window'] = rx_window
else:
    config = raw.config
    
# config = copy.deepcopy(raw.config)
# config['GENERATE']['lo_offset_sw'] = 12.5e6

chirp_ts, ref_chirp = generate_chirp(config)

In [ ]:
# plot reference chirp
fig, axs = plt.subplots(2,1)

# Time domain plot
axs[0].plot(chirp_ts*1e6, np.real(ref_chirp), label='I')
axs[0].plot(chirp_ts*1e6, np.imag(ref_chirp), label='Q')
axs[0].set_xlabel('Time [us]')
axs[0].set_ylabel('Samples')
axs[0].set_title('Time Domain')
axs[0].legend()

# Frequency domain plot
freqs = scipy.fft.fftshift(scipy.fft.fftfreq(ref_chirp.size, d=1/raw.config['GENERATE']['sample_rate']))
ms = 20*np.log10(scipy.fft.fftshift(np.abs(scipy.fft.fft(ref_chirp))))
axs[1].plot(freqs/1e6, ms)
axs[1].set_xlabel('Frequency [MHz]')
axs[1].set_ylabel('Amplitude [dB]')
axs[1].set_title('Frequency Domain')
axs[1].grid()

fig.tight_layout()

plt.show()

### View raw pulse in time domain to check for clipping

In [ ]:
pulse_idx = 2 # raw pulse index you want to look at
plot1 = np.real(raw.radar_data[:,pulse_idx]).hvplot.line(x='fast_time', label='Real') * np.imag(raw.radar_data[:,pulse_idx]).hvplot.line(x='fast_time', label='Imaginary')

plot1 = plot1.opts(xlabel='Fast Time (s)', ylabel='Raw Amplitude')
plot1

### Clean and stack data

In [ ]:
# pr.fill_errors makes errored pulses zeros
# pr.remove_errors takes errored pulses out of the record
# you can choose which one you want to use; if you remove errors, be sure to keep track of how that affects the pulse positioning

#stacked = pr.fill_errors(raw, error_fill_value=0.0, file_error_type="error_data_included") # fill receiver errors with 0s
stacked = pr.remove_errors(raw, file_error_type="error_data_included", skip_if_already_complete=False)

stacked = pr.stack(stacked, nstack) # stack 

### Filter LO out of reference chirp and received data (optional)

In [ ]:
# filter LO out of reference chirp
filter = signal.butter(1, 0.5e6, btype='highpass', output='sos', fs=config['GENERATE']['sample_rate'])

ref_chirp = signal.sosfilt(filter, ref_chirp)

In [ ]:
# filter LO out of received data
filt_hp = signal.butter(1, 0.5e6, btype='highpass', output='sos', fs=config['GENERATE']['sample_rate'])

filtered = xr.apply_ufunc(
    lambda x: signal.sosfilt(filt_hp, x),
    stacked,
    dask="parallelized"
)

In [ ]:
# Plot magnitude response of the filter
freqs_filt, h = signal.sosfreqz(filt_hp, fs=config['GENERATE']['sample_rate'])

fig = plt.figure()
plt.plot(freqs_filt/(1e6), 20 * np.log10(abs(h)),
         'r', label='Bandpass filter', linewidth='2')
plt.xlabel('Frequency [MHz]', fontsize=20)
plt.ylabel('Magnitude [dB]', fontsize=20)
plt.title('Filter', fontsize=20)
plt.grid()

### Pulse compress data

In [ ]:
compressed = pr.pulse_compress(stacked, ref_chirp,
                               fs=stacked.config['GENERATE']['sample_rate'],
                               zero_sample_idx=zero_sample_idx,
                               signal_speed=sig_speed)

compressed_power = xr.apply_ufunc(
    lambda x: 20*np.log10(np.abs(x)),
    compressed,
    dask="parallelized"
)

### View 1D pulse compressed data

In [ ]:
# view pulse compressed data of 1st and last pulse
plot1D = compressed_power.radar_data[0,:].compute().hvplot.line()
plot1D = plot1D * compressed_power.radar_data[-1,:].hvplot.line()
# relevant options: xlim(-80,1000)

plot1D = plot1D.opts(xlabel='Reflection Distance (m)', ylabel='Return Power (dB)', title=raw.basename)
#plot1D.opts(xlim=(-50,200), ylim=(-120, -40), show_grid=True)
plot1D

In [ ]:
# for anna debugging
#import scipy.io

#filename = f"/Users/abroome/Desktop/McMurdo2022/12042022_anna/{raw.basename}_pulsecompressed.mat"
#data_dict = {"pc": compressed_power.radar_data, "dist": compressed_power.reflection_distance}
#scipy.io.savemat(filename, mdict=data_dict)

In [ ]:
# USING MATPLOTLIB
fig, ax = plt.subplots(1,1, figsize=(10,6), facecolor='white')
ax.plot(compressed_power.reflection_distance, compressed_power.radar_data[0,:])
ax.set_xlabel('Depth (m)')
ax.set_ylabel('Return Power (dB)')
ax.set_title(raw.basename)

#fig.savefig(f'/Users/abroome/Desktop/Summit2023/figs/{raw.basename}_pulse_compressed_n{nstack}.png', dpi=300)

### View 2D pulse compressed data (radargram)

In [ ]:
# USING HOLOVIEWS (sometimes breaks)
plot2D = compressed_power.swap_dims({'pulse_idx': 'slow_time', 'travel_time': 'reflection_distance'}).hvplot.quadmesh(x='slow_time', cmap='inferno', flip_yaxis=True)
# relevant options: ylim=(100,-50), clim=(-90,-40)

plot2D.opts(xlabel='Slow Time (s)', ylabel='Depth (m)', clabel='Return Power (dB)')
#plot2D.opts(ylim=(-10, 70), clim=(-120, -40))
plot2D

In [ ]:
# USING MATPLOTLIB (sometimes takes a while)
fig, ax = plt.subplots(1,1, figsize=(18,6), facecolor='white')

p = ax.pcolormesh(compressed_power.slow_time, compressed_power.reflection_distance, compressed_power.radar_data.transpose(), shading='auto', cmap='inferno', vmin=-90, vmax=-10)
ax.invert_yaxis()
clb = fig.colorbar(p, ax=ax)
clb.set_label('Return Power (dB)')
ax.set_xlabel('Slow Time (s)')
ax.set_ylabel('Distance to Reflector (m)')
ax.set_title(f'{raw.basename}: nstack={nstack}')
# relevant options: ax.set_ylim(100,-50), ax.set_xlim(0, 1), vmin=-90, vmax=40
ax.set_ylim(400,-50)

In [ ]:
fig.savefig(f"/Users/abroome/Desktop/McMurdo2022/figs/{raw.basename}_python.png", dpi=300)